# VAZHI SFT v3.3 - Clean Training Notebook

**Key fixes:**
1. FP32 training (Qwen3 has internal bf16 ops that P100/T4 can't handle)
2. `torch_dtype=torch.float16` for model loading
3. ChatML-only data (no raw text mixing)
4. SKIP_DATA_PREP logic to avoid redundant extraction
5. Single GPU forced

**Target:** Kaggle P100 (16GB)

## 1. Install Dependencies

**IMPORTANT:** After running this cell, **RESTART the session** (Runtime → Restart session)

In [ ]:
# Install dependencies
!pip install -q -U \
  "transformers>=4.51.0" \
  "accelerate>=0.34.2" \
  "peft>=0.12.0" \
  "trl>=0.12.0" \
  "bitsandbytes>=0.43.3" \
  "datasets>=2.21.0" \
  "huggingface_hub>=0.24.7"

print("✅ Dependencies installed")
print("⚠️ RESTART THE SESSION NOW (Runtime → Restart session)")

## 2. Imports & Configuration

In [ ]:
# Force single GPU BEFORE importing torch
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import json
import random
import re
import torch
from collections import defaultdict
from datasets import load_dataset, Dataset
from tqdm.auto import tqdm
from huggingface_hub import login, HfApi, dataset_info

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig

# Config
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

# Repos
EXISTING_DATASET = "CryptoYogi/vazhi-tamil-v05"
BALANCED_DATASET = "CryptoYogi/vazhi-tamil-sft-v3_3"
BASE_MODEL = "Qwen/Qwen3-0.6B"
OUTPUT_MODEL = "CryptoYogi/vazhi-qwen3-v3_3"

# System prompt
SYSTEM_PROMPT = "நீங்கள் VAZHI (வழி), தமிழ் மக்களுக்கான AI உதவியாளர். தமிழில் தெளிவாகவும் உதவியாகவும் பதிலளியுங்கள். தெரியாவிட்டால் \"தெரியவில்லை\" என்று சொல்லுங்கள்."

print(f"✅ Configuration loaded")
print(f"   PyTorch: {torch.__version__}")
print(f"   CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
print(f"   Source dataset: {EXISTING_DATASET}")
print(f"   Balanced dataset: {BALANCED_DATASET}")
print(f"   Base model: {BASE_MODEL}")

In [ ]:
# Login to HuggingFace
from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()
hf_token = secrets.get_secret("HF_TOKEN")
login(token=hf_token)
print("✅ Logged in to HuggingFace")

## 3. Helper Functions

In [ ]:
def count_tamil_chars(text):
    """Count Tamil Unicode characters."""
    return sum(1 for c in text if '\u0B80' <= c <= '\u0BFF')

def is_good_tamil_sample(text, min_tamil_pct=30, min_len=10):
    """Check if text has enough Tamil content."""
    if not text or len(text) < min_len:
        return False
    tamil_pct = 100 * count_tamil_chars(text) / len(text)
    return tamil_pct >= min_tamil_pct

def clean_text(text):
    """Clean text by removing extra whitespace."""
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def is_chatml_formatted(text):
    """Check if text is in ChatML format."""
    return '<|im_start|>' in text and '<|im_end|>' in text

def is_kural(text):
    """Check if text contains Thirukkural references."""
    kural_markers = ['குறள்', 'திருக்குறள்', 'kural', 'thirukkural', 'அதிகாரம்']
    text_lower = text.lower()
    return any(m in text_lower for m in kural_markers)

def to_chatml(instruction, output):
    """Convert instruction/output to ChatML format."""
    return f"""<|im_start|>system
{SYSTEM_PROMPT}<|im_end|>
<|im_start|>user
{instruction}<|im_end|>
<|im_start|>assistant
{output}<|im_end|>"""

print("✅ Helper functions defined")

## 4. Check if Dataset Already Exists

In [ ]:
# Check if balanced dataset already exists
SKIP_DATA_PREP = False

try:
    info = dataset_info(BALANCED_DATASET)
    print(f"✅ Dataset {BALANCED_DATASET} already exists!")
    print(f"   Created: {info.created_at}")
    print(f"\n🚀 SKIPPING data preparation - will load directly for training")
    SKIP_DATA_PREP = True
except Exception as e:
    print(f"📝 Dataset {BALANCED_DATASET} not found. Will create it.")
    SKIP_DATA_PREP = False

## 5. Data Preparation (Skip if dataset exists)

In [ ]:
if not SKIP_DATA_PREP:
    def extract_from_indicaling(config_name, max_samples):
        """Extract Tamil samples from IndicAlign."""
        print(f"\n📚 Loading {config_name}...")
        try:
            ds = load_dataset("ai4bharat/indic-align", config_name, split="train", streaming=True)
        except Exception as e:
            print(f"   ⚠️ Error: {e}")
            return []
        
        samples = []
        seen = set()
        
        for item in tqdm(ds, desc=config_name, total=max_samples*5):
            if len(samples) >= max_samples:
                break
            
            tamil = item.get('tam_Taml', [])
            if not tamil or not isinstance(tamil, list) or len(tamil) == 0:
                continue
            
            turns = tamil[0]
            if not isinstance(turns, list) or len(turns) < 2:
                continue
            
            user_msg = clean_text(str(turns[0]))
            assistant_msg = clean_text(str(turns[1]))
            
            if not is_good_tamil_sample(user_msg) or not is_good_tamil_sample(assistant_msg):
                continue
            
            key = user_msg[:100]
            if key in seen:
                continue
            seen.add(key)
            
            samples.append({"instruction": user_msg, "output": assistant_msg, "source": config_name})
        
        print(f"   ✅ Extracted {len(samples)} samples")
        return samples
    
    print("✅ Extraction function defined")
else:
    print("⏭️ Skipping - dataset exists")

In [ ]:
if not SKIP_DATA_PREP:
    print("🚀 Extracting from IndicAlign...")
    diverse_samples = []
    diverse_samples.extend(extract_from_indicaling("Dolly_T", 300))
    diverse_samples.extend(extract_from_indicaling("WikiHow", 250))
    diverse_samples.extend(extract_from_indicaling("Wiki_Conv", 300))
    diverse_samples.extend(extract_from_indicaling("OpenAssistant_T", 200))
    print(f"\n📊 Total from IndicAlign: {len(diverse_samples)}")
else:
    print("⏭️ Skipping IndicAlign extraction")

In [ ]:
if not SKIP_DATA_PREP:
    # Manual samples for short answers and behavior
    manual_samples = [
        # Geography
        {"instruction": "தமிழ்நாட்டின் தலைநகரம் என்ன?", "output": "சென்னை.", "source": "manual"},
        {"instruction": "இந்தியாவின் தலைநகரம் எது?", "output": "புது தில்லி.", "source": "manual"},
        {"instruction": "தமிழ்நாட்டின் மாவட்டங்கள் எத்தனை?", "output": "38 மாவட்டங்கள்.", "source": "manual"},
        
        # Basic facts
        {"instruction": "சூரியன் எந்த திசையில் உதிக்கும்?", "output": "கிழக்கு திசையில்.", "source": "manual"},
        {"instruction": "ஒரு வாரத்தில் எத்தனை நாட்கள்?", "output": "ஏழு நாட்கள்.", "source": "manual"},
        {"instruction": "2+2 என்ன?", "output": "4.", "source": "manual"},
        {"instruction": "10 x 10 என்ன?", "output": "100.", "source": "manual"},
        
        # Tamil culture
        {"instruction": "பொங்கல் எப்போது கொண்டாடப்படுகிறது?", "output": "தை மாதம் முதல் நாள் (ஜனவரி 14 அல்லது 15).", "source": "manual"},
        {"instruction": "தமிழ் எழுத்துக்கள் எத்தனை?", "output": "247 எழுத்துக்கள்.", "source": "manual"},
        {"instruction": "சிலப்பதிகாரத்தை எழுதியவர் யார்?", "output": "இளங்கோவடிகள்.", "source": "manual"},
        
        # Behavior
        {"instruction": "வணக்கம்", "output": "வணக்கம்! நான் வழி. உங்களுக்கு எப்படி உதவ வேண்டும்?", "source": "behavior"},
        {"instruction": "hi", "output": "வணக்கம்! எப்படி உதவலாம்?", "source": "behavior"},
        {"instruction": "2050-ல் யார் பிரதமர் ஆவார்?", "output": "எதிர்காலத்தை கணிக்க என்னால் முடியாது. தெரியவில்லை.", "source": "behavior"},
        {"instruction": "என்னுடைய கடவுச்சொல் என்ன?", "output": "உங்கள் தனிப்பட்ட தகவல்கள் என்னிடம் இல்லை.", "source": "behavior"},
        {"instruction": "நன்றி", "output": "மகிழ்ச்சி! வேறு உதவி தேவைப்பட்டால் கேளுங்கள்.", "source": "behavior"},
    ]
    
    diverse_samples.extend(manual_samples)
    print(f"📊 Total after manual samples: {len(diverse_samples)}")
else:
    print("⏭️ Skipping manual samples")

In [ ]:
if not SKIP_DATA_PREP:
    print(f"\n📚 Loading existing dataset from {EXISTING_DATASET}...")
    existing_ds = load_dataset(EXISTING_DATASET, split="train")
    print(f"   Loaded {len(existing_ds)} samples")
    
    # Filter for ChatML ONLY
    existing_kural_chatml = []
    existing_other_chatml = []
    
    for item in tqdm(existing_ds, desc="Filtering ChatML"):
        text = item.get('text', '')
        if is_chatml_formatted(text):
            if is_kural(text):
                existing_kural_chatml.append({"text": text})
            else:
                existing_other_chatml.append({"text": text})
    
    print(f"\n📊 ChatML samples:")
    print(f"   Kural: {len(existing_kural_chatml)}")
    print(f"   Other: {len(existing_other_chatml)}")
else:
    print("⏭️ Skipping existing dataset loading")

In [ ]:
if not SKIP_DATA_PREP:
    # Downsample Thirukkural to ~25%
    total_other = len(existing_other_chatml)
    target_kural_pct = 0.25
    target_kural_count = int(target_kural_pct * total_other / (1 - target_kural_pct))
    
    if len(existing_kural_chatml) > target_kural_count:
        downsampled_kural = random.sample(existing_kural_chatml, target_kural_count)
    else:
        downsampled_kural = existing_kural_chatml
    
    print(f"🎯 Downsampled Kural: {len(existing_kural_chatml)} → {len(downsampled_kural)}")
    
    # Convert diverse to ChatML and combine
    diverse_formatted = [{"text": to_chatml(s["instruction"], s["output"])} for s in diverse_samples]
    
    final_samples = []
    final_samples.extend(downsampled_kural)
    final_samples.extend(existing_other_chatml)
    final_samples.extend(diverse_formatted)
    random.shuffle(final_samples)
    
    print(f"\n📊 Final dataset: {len(final_samples)} samples")
    
    # Verify 100% ChatML
    chatml_count = sum(1 for s in final_samples if is_chatml_formatted(s["text"]))
    if chatml_count != len(final_samples):
        raise ValueError("Not all samples are ChatML formatted!")
    print(f"✅ 100% ChatML verified")
else:
    print("⏭️ Skipping dataset combination")

In [ ]:
if not SKIP_DATA_PREP:
    # Save and upload
    os.makedirs("/kaggle/working/balanced_sft", exist_ok=True)
    
    split_idx = int(0.95 * len(final_samples))
    train_samples = final_samples[:split_idx]
    val_samples = final_samples[split_idx:]
    
    with open("/kaggle/working/balanced_sft/train.jsonl", 'w') as f:
        for s in train_samples:
            f.write(json.dumps(s, ensure_ascii=False) + '\n')
    
    with open("/kaggle/working/balanced_sft/val.jsonl", 'w') as f:
        for s in val_samples:
            f.write(json.dumps(s, ensure_ascii=False) + '\n')
    
    print(f"💾 Saved: {len(train_samples)} train, {len(val_samples)} val")
    
    # Upload to HuggingFace
    api = HfApi()
    api.create_repo(BALANCED_DATASET, repo_type="dataset", exist_ok=True)
    api.upload_file(
        path_or_fileobj="/kaggle/working/balanced_sft/train.jsonl",
        path_in_repo="train.jsonl",
        repo_id=BALANCED_DATASET,
        repo_type="dataset"
    )
    api.upload_file(
        path_or_fileobj="/kaggle/working/balanced_sft/val.jsonl",
        path_in_repo="val.jsonl",
        repo_id=BALANCED_DATASET,
        repo_type="dataset"
    )
    print(f"✅ Uploaded to {BALANCED_DATASET}")
else:
    print("⏭️ Skipping upload")

## 6. Load Dataset for Training

In [ ]:
print(f"📚 Loading balanced dataset...")
balanced_ds = load_dataset(BALANCED_DATASET, split="train")
print(f"✅ Loaded {len(balanced_ds)} samples")

# Verify ChatML format
sample = balanced_ds[0]['text'][:200]
print(f"\n📝 Sample: {sample}...")
if "<|im_start|>" in sample:
    print("✅ ChatML format verified")

## 7. Load Model with 4-bit Quantization

In [ ]:
print("\n📥 Loading model and tokenizer...")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.padding_side = "right"

# 4-bit quantization - use float16 compute dtype
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load model - MUST specify torch_dtype to avoid bf16 default
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map={"":0},
    trust_remote_code=True
)

# Prepare for training
model = prepare_model_for_kbit_training(model)
model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id
model.config.use_cache = False

print(f"✅ Model loaded: {model.num_parameters():,} params")
print(f"   torch_dtype: float16")

## 8. Add LoRA Adapters

In [ ]:
# LoRA config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Verify no bf16 parameters
bf16_count = sum(1 for _, p in model.named_parameters() if p.dtype == torch.bfloat16)
if bf16_count > 0:
    print(f"⚠️ Found {bf16_count} bf16 parameters - converting to fp16")
    for name, param in model.named_parameters():
        if param.dtype == torch.bfloat16:
            param.data = param.data.to(torch.float16)
else:
    print("✅ No bf16 parameters")

## 9. Training (FP32 Mode for P100 Compatibility)

In [ ]:
# FP32 training - Qwen3 has internal bf16 ops that P100 can't handle with AMP
sft_config = SFTConfig(
    output_dir="/kaggle/working/vazhi-v3_3",
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    logging_steps=25,
    save_steps=200,
    save_total_limit=2,
    fp16=False,  # DISABLED - Qwen3 has internal bf16
    bf16=False,
    gradient_checkpointing=True,
    max_grad_norm=1.0,
    optim="paged_adamw_8bit",
    report_to="none",
    dataset_text_field="text",
    max_length=512,
    packing=False,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=balanced_ds,
    args=sft_config,
    processing_class=tokenizer,
)

print("✅ Trainer initialized (FP32 mode)")
print(f"   Epochs: 2")
print(f"   Batch size: 1 x 16 = 16 effective")
print(f"   Mode: FP32 (P100 compatible)")

In [ ]:
# Train!
print("\n🚀 Starting training...")
trainer.train()
print("\n✅ Training complete!")

## 10. Save and Push to HuggingFace

In [ ]:
print("💾 Saving model...")
trainer.save_model("/kaggle/working/vazhi-v3_3-final")

print("🔀 Merging LoRA weights...")
merged_model = model.merge_and_unload()

# Push to HuggingFace
api = HfApi()
api.create_repo(OUTPUT_MODEL, exist_ok=True)

print(f"📤 Pushing to {OUTPUT_MODEL}...")
merged_model.push_to_hub(OUTPUT_MODEL, private=False)
tokenizer.push_to_hub(OUTPUT_MODEL, private=False)

print(f"\n✅ Model uploaded: https://huggingface.co/{OUTPUT_MODEL}")

## 11. Test the Model

In [ ]:
merged_model.config.use_cache = True

test_prompts = [
    "வணக்கம்",
    "தமிழ்நாட்டின் தலைநகரம் என்ன?",
    "2+2 என்ன?",
    "பொங்கல் எப்போது கொண்டாடப்படுகிறது?",
]

print("\n🧪 Testing model...\n")

for prompt in test_prompts:
    full_prompt = f"<|im_start|>system\n{SYSTEM_PROMPT}<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"
    
    inputs = tokenizer(full_prompt, return_tensors="pt").to(merged_model.device)
    
    with torch.no_grad():
        outputs = merged_model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.5,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.3,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    if "<|im_start|>assistant" in response:
        response = response.split("<|im_start|>assistant")[-1]
        response = response.split("<|im_end|>")[0].strip()
    
    print(f"Q: {prompt}")
    print(f"A: {response}")
    print("-" * 50)

## Summary

- **FP32 training** - Qwen3 has internal bf16 ops incompatible with P100
- **ChatML only** - No raw text mixing
- **Thirukkural ~25%** - Downsampled from 71%
- **4-bit QLoRA** - Memory efficient